# Data wrangling

Add imports

In [3]:
import os
import pandas as pd
import re

Read raw output .csv's from obtained by crawling. Add new columns `Search word` and `Most watched`

In [4]:
folder_path = "../outputs/"
dataframes = []
pattern = r"(.+?)(?:_(most_watched))?\.csv"

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):

        match = re.search(pattern, filename)
        if match:
            word = match.group(1)
            most_watched = match.group(2) is not None
            file_path = os.path.join(folder_path, filename)
            file_df = pd.read_csv(file_path)
            file_df["Search word"] = word
            file_df["Most watched"] = most_watched
            dataframes.append(file_df)

df = pd.concat(dataframes, ignore_index=True)
df.head()

,Video Id,Title,View Count,Length,Channel,Search word,Most watched
0,ssFgYNn0BUA,Mallrat - Groceries (Official Video),2 900 865 katselukertaa,3.35,/channel/UCK2codDA94XHf66dbfSy6vw,groceries,False
1,34djOMgVq-0,Pi'erre Bourne - Groceries (Official Music Video),1 644 500 katselukertaa,4.10,/channel/UCN27za3wzItmyRV2O3N91vw,groceries,False
2,ha4tRQwKIUg,I Bought Everything In A Store - Challenge,144 516 730 katselukertaa,16.56,/@MrBeast,groceries,False
3,8oZNzedMhUo,"Roommate WON'T BUY Groceries, What Happens Is ...",20 756 555 katselukertaa,8.14,/@DharMann,groceries,False
4,WClN_VrYosc,HOW WE WENT BROKE GROCERY SHOPPING FOR FOOD **...,721 509 katselukertaa,40.56,/@TheOfficialPrinceFamily,groceries,False


Remove NaN's and create new column `Views` as type `int`

In [5]:
df.dropna(subset=["View Count", "Length"], inplace=True)
df["Views"] = df["View Count"].str.extract("(\d+(?:\s\d+)*)").replace(r"\s+", "", regex=True).fillna("0").astype(int)
df.describe()

,Views
count,7.374190e+05
mean,5.084356e+07
std,2.158456e+08
min,0.000000e+00
25%,2.085415e+05
50%,3.095021e+06
75%,2.633310e+07
max,1.335517e+10


Change `Search word` and `Most watched` types

In [6]:
df["Search word"] = df["Search word"].apply(lambda x: [x])
df["Most watched"] = df["Most watched"].map({True: "Y", False: "N"})
df.head()

,Video Id,Title,View Count,Length,Channel,Search word,Most watched,Views
0,ssFgYNn0BUA,Mallrat - Groceries (Official Video),2 900 865 katselukertaa,3.35,/channel/UCK2codDA94XHf66dbfSy6vw,[groceries],N,2900865
1,34djOMgVq-0,Pi'erre Bourne - Groceries (Official Music Video),1 644 500 katselukertaa,4.10,/channel/UCN27za3wzItmyRV2O3N91vw,[groceries],N,1644500
2,ha4tRQwKIUg,I Bought Everything In A Store - Challenge,144 516 730 katselukertaa,16.56,/@MrBeast,[groceries],N,144516730
3,8oZNzedMhUo,"Roommate WON'T BUY Groceries, What Happens Is ...",20 756 555 katselukertaa,8.14,/@DharMann,[groceries],N,20756555
4,WClN_VrYosc,HOW WE WENT BROKE GROCERY SHOPPING FOR FOOD **...,721 509 katselukertaa,40.56,/@TheOfficialPrinceFamily,[groceries],N,721509


Group by `Video Id` and aggregate `Search words` and `Most watched` information

In [7]:
def append_words(series):
    return list(set([item for sublist in series for item in sublist]))

def append_search_type(series):
    return set(series)

aggregations = {
    "Title": "first",
    "Views": "first",
    "Length": "first",
    "Channel": "first",
    "Search word": append_words,
    "Most watched": append_search_type
}

df = df.groupby("Video Id").agg(aggregations).reset_index()

Convert `Most watched` into three categorial values: ```Y = Most watched | N = Normal | B = Both```

In [8]:
def convert_most_watched(values):
    if values == {"N", "Y"}:
        return "B"
    elif "Y" in values:
        return "Y"
    else:
        return "N"

df["Most watched"] = df["Most watched"].apply(convert_most_watched).astype("category")

Result checking head and tail

In [9]:
df.head()

,Video Id,Title,Views,Length,Channel,Search word,Most watched
0,---AKxmFRWI,Dharmesh Yalande Sir Dance Bastar Dist Chhatti...,8867396,0.27,/@bastarfiles,[dist],Y
1,---jp8dVvkY,Fed Will Likely Hike in September - Market Pul...,144,26.55,/@FXPesa,[pulse],N
2,--0HXqi8xTk,Hyundai i20 at 25% Discount,798201,0.59,/@bekifaayati,[discounted],Y
3,--0Qq5EmpGg,PSY - GANGNAM STYLE (CONVERSELY),424,4.13,/@maxchet,[conversely],N
4,--14w5SOEUs,Migos - Avalanche (Official Video),21229658,3.59,/channel/UC9YcTIQuhwgoOQqYMKYqW9A,[avalanche],Y


In [10]:
df.tail()

,Video Id,Title,Views,Length,Channel,Search word,Most watched
524554,zzwWzLY7kEo,NewJeans (뉴진스) 'ETA' Dance Practice,3131516,2.39,/channel/UCMki_UkHb4qSc0qyEcOHHJw,[eta],N
524555,zzwXTkQ9n0k,Latest trend with Horse chestnut #asmr #oddlys...,68243230,0.19,/@hoofmaestro1130,[chestnut],Y
524556,zzwmEo-mzDc,Meryl - La Brume ft. Le Motif,5000660,3.13,/@MERYLytchn,[motif],Y
524557,zzyDz6jhX30,All I Ever Need - Austin Mahone (Lyrics),13613316,3.34,/@SuperbLyricsOfficial,[necessity],Y
524558,zzzIpC39WUg,How To Calculate Speeds and Feeds (Inch Versio...,267679,14.26,/@haasautomation,[feeds],N


Display videos without views.

In [11]:
zero_views = df[df['Views'] == 0]
display(zero_views)

,Video Id,Title,Views,Length,Channel,Search word,Most watched
11297,0OcrOLPi2DY,"Recent Jazz Vinyl Finds #7 - Tons of grails, B...",0,37.35,/@amanandhismusic,[mccoy],N
14206,0kR1d1maUqM,Basisschool Het Christal in Vlaardingen (LEV-WN),0,1.26,/@LEV-ScholengroepWest-Nederland,[wn],N
25938,2APJAThtBzE,FF7 Ever Crisis Pulls for Aerith's Sunny Robe!,0,13.43,/@lad1174,[robe],N
28380,2Twncb3dE_o,BROKEN TO BEAST the Ultimate Warrior Overland ...,0,13.00,/@WolfPacOutdoors,[overland],N
30499,2jQyYTCkIXU,terminals specification viewpicture end italia...,0,1.41,/@phillipsandie6581,[viewpicture],Y
...,...,...,...,...,...,...,...
514039,yhFsW8RBwRE,NEW THIRD GEN!!! (IT BLEW UP ONLY HAD IR FOR T...,0,10.15,/@FBody_Addicts,[blew],N
515532,ysnYuGZG-zE,How to Whip Whipy Whip Whipping Cream Perfectl...,0,5.04,/@VillageKitchenGarden,[whipping],N
517375,z6WSeB7fMoo,Personally Speaking ep. 172 (Billy Keenan),0,28.00,/@PersonallySpeakingwithMsgrJimL,[personally],N
518265,zDQH7Yo0Um0,Eriette of the Charmed Apple $40 BUDGET EDH De...,0,9.51,/@GatheringTheMagicYT,[charmed],N


Remove music videos and filter non latin character containing titles.

In [12]:
words = ["vevo", "lyric", "music", "song", "official video", "mix", "remix" "album", "feat", "instrumental", "ft."]

def filter_words(title):
  low_title = title.lower()
  for w in words:
    if w in low_title:
      return False
  return True

filtered_df = df[df["Title"].apply(filter_words) & df["Channel"].apply(filter_words)]



In [13]:

print("unfiltered df: ", df.shape)
print("filtered df: ", filtered_df.shape)
print(f"row diff: -{df.shape[0] - filtered_df.shape[0]}\nlost {100 - round(filtered_df.shape[0] / df.shape[0] * 100)}% of rows")
display(filtered_df)

# -18% for just music words, -26% for music words AND # symbol

unfiltered df:  (524559, 7)
filtered df:  (443694, 7)
row diff: -80865
lost 15% of rows


,Video Id,Title,Views,Length,Channel,Search word,Most watched
0,---AKxmFRWI,Dharmesh Yalande Sir Dance Bastar Dist Chhatti...,8867396,0.27,/@bastarfiles,[dist],Y
1,---jp8dVvkY,Fed Will Likely Hike in September - Market Pul...,144,26.55,/@FXPesa,[pulse],N
2,--0HXqi8xTk,Hyundai i20 at 25% Discount,798201,0.59,/@bekifaayati,[discounted],Y
3,--0Qq5EmpGg,PSY - GANGNAM STYLE (CONVERSELY),424,4.13,/@maxchet,[conversely],N
5,--1bxl5E-u8,Firmly I Believe and Truly God is Three (Halto...,5143,1.29,/@AndrewRemillard,[firmly],N
...,...,...,...,...,...,...,...
524552,zzwRbKI2pn4,YALGAAR - CARRYMINATI X Wily Frenzy,350657873,3.15,/@CarryMinati,"[carry, younger, frenzy]",Y
524553,zzwUH3vbNkc,Klassendiagramme mit UML - Theoretische Objekt...,286980,7.32,/@simpleclub_informatik,[uml],Y
524554,zzwWzLY7kEo,NewJeans (뉴진스) 'ETA' Dance Practice,3131516,2.39,/channel/UCMki_UkHb4qSc0qyEcOHHJw,[eta],N
524555,zzwXTkQ9n0k,Latest trend with Horse chestnut #asmr #oddlys...,68243230,0.19,/@hoofmaestro1130,[chestnut],Y


### Create new column for original titles. Stopwords, punctuations, non-latin words, stemming and capital word counting

Create new column for original titles

In [14]:
filtered_df = filtered_df.copy()
filtered_df.insert(filtered_df.columns.get_loc("Title"), "Original Title", filtered_df["Title"])


Remove punctuations

In [15]:
filtered_df["Title"] = filtered_df["Title"].str.replace(r"[^\w\s]+", "", regex=True)
filtered_df.head()

,Video Id,Original Title,Title,Views,Length,Channel,Search word,Most watched
0,---AKxmFRWI,Dharmesh Yalande Sir Dance Bastar Dist Chhatti...,Dharmesh Yalande Sir Dance Bastar Dist Chhatti...,8867396,0.27,/@bastarfiles,[dist],Y
1,---jp8dVvkY,Fed Will Likely Hike in September - Market Pul...,Fed Will Likely Hike in September Market Puls...,144,26.55,/@FXPesa,[pulse],N
2,--0HXqi8xTk,Hyundai i20 at 25% Discount,Hyundai i20 at 25 Discount,798201,0.59,/@bekifaayati,[discounted],Y
3,--0Qq5EmpGg,PSY - GANGNAM STYLE (CONVERSELY),PSY GANGNAM STYLE CONVERSELY,424,4.13,/@maxchet,[conversely],N
5,--1bxl5E-u8,Firmly I Believe and Truly God is Three (Halto...,Firmly I Believe and Truly God is Three Halton...,5143,1.29,/@AndrewRemillard,[firmly],N


Remove non-latin words and words with numbers etc.

In [16]:
filtered_df["Title"] = filtered_df["Title"].str.replace(r"\b(?![a-zA-Z]+\b)\w+\b", "", regex=True)
display(filtered_df)

,Video Id,Original Title,Title,Views,Length,Channel,Search word,Most watched
0,---AKxmFRWI,Dharmesh Yalande Sir Dance Bastar Dist Chhatti...,Dharmesh Yalande Sir Dance Bastar Dist Chhatti...,8867396,0.27,/@bastarfiles,[dist],Y
1,---jp8dVvkY,Fed Will Likely Hike in September - Market Pul...,Fed Will Likely Hike in September Market Puls...,144,26.55,/@FXPesa,[pulse],N
2,--0HXqi8xTk,Hyundai i20 at 25% Discount,Hyundai at Discount,798201,0.59,/@bekifaayati,[discounted],Y
3,--0Qq5EmpGg,PSY - GANGNAM STYLE (CONVERSELY),PSY GANGNAM STYLE CONVERSELY,424,4.13,/@maxchet,[conversely],N
5,--1bxl5E-u8,Firmly I Believe and Truly God is Three (Halto...,Firmly I Believe and Truly God is Three Halton...,5143,1.29,/@AndrewRemillard,[firmly],N
...,...,...,...,...,...,...,...,...
524552,zzwRbKI2pn4,YALGAAR - CARRYMINATI X Wily Frenzy,YALGAAR CARRYMINATI X Wily Frenzy,350657873,3.15,/@CarryMinati,"[carry, younger, frenzy]",Y
524553,zzwUH3vbNkc,Klassendiagramme mit UML - Theoretische Objekt...,Klassendiagramme mit UML Theoretische Objekto...,286980,7.32,/@simpleclub_informatik,[uml],Y
524554,zzwWzLY7kEo,NewJeans (뉴진스) 'ETA' Dance Practice,NewJeans ETA Dance Practice,3131516,2.39,/channel/UCMki_UkHb4qSc0qyEcOHHJw,[eta],N
524555,zzwXTkQ9n0k,Latest trend with Horse chestnut #asmr #oddlys...,Latest trend with Horse chestnut asmr oddlysat...,68243230,0.19,/@hoofmaestro1130,[chestnut],Y


Check and remove if there are empty Title's due above operation and remove from the list (ie. fully non-latin titles etc.)

In [39]:
#Store length before operation
filtered_df_length = len(filtered_df)

# Used to see what kind of Original Title's belongs to this category
non_latin_df = filtered_df[filtered_df["Title"] == ""]

# Do the filtering
filtered_df = filtered_df[filtered_df["Title"] != ""]

print(f"Former length: {filtered_df_length} | New length: {len(filtered_df)} | Non-latins removed: {len(non_latin_df)} ({len(non_latin_df) / filtered_df_length * 100:1f}%)")
non_latin_df.head()


Former length: 443015 | New length: 439504 | Non-latins removed: 3511 (0.792524%)


,Video Id,Original Title,Title,Views,Length,Channel,Search word,Most watched,Uppercase Words,Lowercase Words,Capitalized Words,Word Count
40,--FmExEAsM8,IVE 아이브 'ELEVEN' MV,,203338937,3.04,/@STARSHIP_official,"[iv, eleven, ive, ivy]",B,3,0,0,4
114,--nlCyTbADU,စမ်းချင်ရာစမ်း မောင်ဘဂျမ်း/ စမ္းခ်င္ရာစမ္း ေမာ...,,6565004,1.03.40,/@LaurelArtAcademy,[laurel],Y,0,0,0,18
372,-23UKjwu_10,Where Are You Now,,36940440,2.29,/channel/UCdKS_mDSLUkS6vDK6u1mjOg,[frequencies],Y,0,0,4,4
466,-2tUSpRS_fk,ماهذا الجيل ؟ 🤷‍♂️😒,,23690,5.23,/@CrankTN,[crank],N,0,0,0,2
739,-4y9CUgpjUo,A a a a a a a a a u a a a a,,19520338,0.22,/@FrostDiamond,[frost],Y,1,13,1,14


Save uppercased word count, lower case word count and words that start with capital character e.g 'Cat'.

In [40]:
def word_counts(row):
  sentence = row['Original Title']
  uppercase_pattern = r'\b(?:[A-Z]+\d*|[A-Z]*\d+[A-Z]+)[A-Z0-9]*\b'
  lowercase_pattern = r'\b(?:[a-z]+\d*|[a-z]*\d+[a-z]+)[a-z0-9]*\b'
  capital_pattern = r'\b[A-Z][a-z0-9]*\b'
  word_count_pattern = r'\w+'

  uppercase_count = len(re.findall(uppercase_pattern, sentence))
  lowercase_count = len(re.findall(lowercase_pattern, sentence))
  capital_count = len(re.findall(capital_pattern, sentence))
  word_count = len(re.findall(word_count_pattern, sentence))
  return pd.Series({'Uppercase Words': uppercase_count, 'Lowercase Words': lowercase_count, 'Capitalized Words': capital_count, 'Word Count': word_count})

filtered_df[['Uppercase Words', 'Lowercase Words', 'Capitalized Words', 'Word Count']] = filtered_df.apply(word_counts, axis = 1)

display(filtered_df)


,Video Id,Original Title,Title,Views,Length,Channel,Search word,Most watched,Uppercase Words,Lowercase Words,Capitalized Words,Word Count
0,---AKxmFRWI,Dharmesh Yalande Sir Dance Bastar Dist Chhatti...,dharmesh yaland sir danc bastar dist chhattisg...,8867396,0.27,/@bastarfiles,[dist],Y,0,0,13,13
1,---jp8dVvkY,Fed Will Likely Hike in September - Market Pul...,fed hike septemb market puls rufa kamau,144,26.55,/@FXPesa,[pulse],N,0,2,9,11
2,--0HXqi8xTk,Hyundai i20 at 25% Discount,hyundai discount,798201,0.59,/@bekifaayati,[discounted],Y,0,2,2,5
3,--0Qq5EmpGg,PSY - GANGNAM STYLE (CONVERSELY),psi gangnam style convers,424,4.13,/@maxchet,[conversely],N,4,0,0,4
5,--1bxl5E-u8,Firmly I Believe and Truly God is Three (Halto...,firmli god halton holgat,5143,1.29,/@AndrewRemillard,[firmly],N,1,2,8,10
...,...,...,...,...,...,...,...,...,...,...,...,...
524552,zzwRbKI2pn4,YALGAAR - CARRYMINATI X Wily Frenzy,yalgaar carryminati wili frenzi,350657873,3.15,/@CarryMinati,"[carry, younger, frenzy]",Y,3,0,3,5
524553,zzwUH3vbNkc,Klassendiagramme mit UML - Theoretische Objekt...,klassendiagramm mit uml theoretisch objektorie...,286980,7.32,/@simpleclub_informatik,[uml],Y,1,1,4,7
524554,zzwWzLY7kEo,NewJeans (뉴진스) 'ETA' Dance Practice,newjean eta danc practic,3131516,2.39,/channel/UCMki_UkHb4qSc0qyEcOHHJw,[eta],N,1,0,2,5
524555,zzwXTkQ9n0k,Latest trend with Horse chestnut #asmr #oddlys...,trend hors chestnut asmr oddlysatisfi tiktok f...,68243230,0.19,/@hoofmaestro1130,[chestnut],Y,0,7,2,9


Change to lowercase and remove stop words

In [41]:
filtered_df["Title"] = filtered_df["Title"].str.lower()

with open("stop.txt") as f:
    stopwords = set(f.read().split("\n"))

filtered_df["Title"] = filtered_df["Title"].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stopwords)]))
filtered_df.head()


,Video Id,Original Title,Title,Views,Length,Channel,Search word,Most watched,Uppercase Words,Lowercase Words,Capitalized Words,Word Count
0,---AKxmFRWI,Dharmesh Yalande Sir Dance Bastar Dist Chhatti...,dharmesh yaland sir danc bastar dist chhattisg...,8867396,0.27,/@bastarfiles,[dist],Y,0,0,13,13
1,---jp8dVvkY,Fed Will Likely Hike in September - Market Pul...,fed hike septemb market puls rufa kamau,144,26.55,/@FXPesa,[pulse],N,0,2,9,11
2,--0HXqi8xTk,Hyundai i20 at 25% Discount,hyundai discount,798201,0.59,/@bekifaayati,[discounted],Y,0,2,2,5
3,--0Qq5EmpGg,PSY - GANGNAM STYLE (CONVERSELY),psi gangnam style convers,424,4.13,/@maxchet,[conversely],N,4,0,0,4
5,--1bxl5E-u8,Firmly I Believe and Truly God is Three (Halto...,firmli god halton holgat,5143,1.29,/@AndrewRemillard,[firmly],N,1,2,8,10


Add stemming

In [42]:
from nltk.stem.porter import *

ps = PorterStemmer()
filtered_df["Title"] = filtered_df["Title"].apply(lambda x: ' '.join([ps.stem(word) for word in str(x).split()]))
display(filtered_df)

,Video Id,Original Title,Title,Views,Length,Channel,Search word,Most watched,Uppercase Words,Lowercase Words,Capitalized Words,Word Count
0,---AKxmFRWI,Dharmesh Yalande Sir Dance Bastar Dist Chhatti...,dharmesh yaland sir danc bastar dist chhattisg...,8867396,0.27,/@bastarfiles,[dist],Y,0,0,13,13
1,---jp8dVvkY,Fed Will Likely Hike in September - Market Pul...,fed hike septemb market pul rufa kamau,144,26.55,/@FXPesa,[pulse],N,0,2,9,11
2,--0HXqi8xTk,Hyundai i20 at 25% Discount,hyundai discount,798201,0.59,/@bekifaayati,[discounted],Y,0,2,2,5
3,--0Qq5EmpGg,PSY - GANGNAM STYLE (CONVERSELY),psi gangnam style conver,424,4.13,/@maxchet,[conversely],N,4,0,0,4
5,--1bxl5E-u8,Firmly I Believe and Truly God is Three (Halto...,firmli god halton holgat,5143,1.29,/@AndrewRemillard,[firmly],N,1,2,8,10
...,...,...,...,...,...,...,...,...,...,...,...,...
524552,zzwRbKI2pn4,YALGAAR - CARRYMINATI X Wily Frenzy,yalgaar carryminati wili frenzi,350657873,3.15,/@CarryMinati,"[carry, younger, frenzy]",Y,3,0,3,5
524553,zzwUH3vbNkc,Klassendiagramme mit UML - Theoretische Objekt...,klassendiagramm mit uml theoretisch objektorie...,286980,7.32,/@simpleclub_informatik,[uml],Y,1,1,4,7
524554,zzwWzLY7kEo,NewJeans (뉴진스) 'ETA' Dance Practice,newjean eta danc practic,3131516,2.39,/channel/UCMki_UkHb4qSc0qyEcOHHJw,[eta],N,1,0,2,5
524555,zzwXTkQ9n0k,Latest trend with Horse chestnut #asmr #oddlys...,trend hor chestnut asmr oddlysatisfi tiktok fa...,68243230,0.19,/@hoofmaestro1130,[chestnut],Y,0,7,2,9


Remove titles with 0 views

In [43]:
print(filtered_df.shape)
cleaned_df = filtered_df[filtered_df['Views'] > 0]
print(cleaned_df.shape)

(439504, 12)
(439393, 12)


### Store wrangled dataframe into outputs

In [47]:
#Split DF dues Github size limit
i = len(cleaned_df) // 2

cleaned_df.iloc[:i].to_csv("wrangled_data_part1.csv", index=False, encoding="utf-8")
cleaned_df.iloc[i:].to_csv("wrangled_data_part2.csv", index=False, encoding="utf-8")